In [1]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.bahir:spark-streaming-mqtt_2.11:2.2.1,org.apache.bahir:spark-sql-streaming-mqtt_2.11:2.2.1 pyspark-shell'

In [2]:
from pyspark.storagelevel import StorageLevel
from pyspark.serializers import UTF8Deserializer
from pyspark.streaming import DStream

#   Subscribe to mqtt broker and follow given topics
def createStream(ssc, brokerUrl, topic, storageLevel=StorageLevel.MEMORY_AND_DISK_SER_2):
    jlevel = ssc._sc._getJavaStorageLevel(storageLevel)

    helperClass = ssc._jvm.java.lang.Thread.currentThread().getContextClassLoader() \
        .loadClass("org.apache.spark.streaming.mqtt.MQTTUtilsPythonHelper")
    helper = helperClass.newInstance()
    jstream = helper.createStream(ssc._jssc, brokerUrl, topic, jlevel)

    return DStream(jstream, ssc, UTF8Deserializer())


In [3]:
import sys
import json
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

sc = SparkContext(appName="HumAIn Test")
ssc = StreamingContext(sc, 10)
ssc.checkpoint("checkpoint")

brokerUrl = "tcp://localhost:1883"
topic1 = "humAIn/testTopic1"
mqttPort = 1883

mqttStream1 = createStream(ssc, brokerUrl, topic1)

In [5]:
#   For each row line in rdd
#   Convert text to JSON
#   Enhance JSON
#   Dump JSON to text and put text in line
def enhanceRecord(record):   
    payload = json.loads(record)
    tagset = payload
    return json.dumps(tagset)

mqttStreamEnhanced = mqttStream1.map(lambda x: enhanceRecord(x))

In [7]:
#   For each rdd in newly created joined dstream print output
mqttStreamEnhanced.pprint()

In [8]:
ssc.start()
ssc.awaitTermination()

-------------------------------------------
Time: 2021-11-25 17:42:50
-------------------------------------------
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))

-------------------------------------------
Time: 2021-11-25 17:43:00
-------------------------------------------
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
...

-------------------------------------------
Time: 2021-11-25 17:43:10
-------------------------------------------
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
...

-------------------------------------------
Time: 2021-11-25 17:43:20
-------------------------------------------
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"',

Py4JJavaError: An error occurred while calling o26.awaitTermination.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "C:\Users\mikke\anaconda3\envs\roborana\lib\site-packages\pyspark\streaming\util.py", line 68, in call
    r = self.func(t, *rdds)
  File "C:\Users\mikke\anaconda3\envs\roborana\lib\site-packages\pyspark\streaming\dstream.py", line 173, in takeAndPrint
    taken = rdd.take(num + 1)
  File "C:\Users\mikke\anaconda3\envs\roborana\lib\site-packages\pyspark\rdd.py", line 1360, in take
    res = self.context.runJob(self, takeUpToNumLeft, p)
  File "C:\Users\mikke\anaconda3\envs\roborana\lib\site-packages\pyspark\context.py", line 1069, in runJob
    sock_info = self._jvm.PythonRDD.runJob(self._jsc.sc(), mappedRDD._jrdd, partitions)
  File "C:\Users\mikke\anaconda3\envs\roborana\lib\site-packages\py4j\java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "C:\Us

-------------------------------------------
Time: 2021-11-25 17:45:30
-------------------------------------------
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
...

-------------------------------------------
Time: 2021-11-25 17:45:40
-------------------------------------------
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
...

-------------------------------------------
Time: 2021-11-25 17:45:50
-------------------------------------------
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
('{', ('"', '"'))
...

-------------------------------------------
Time: 2021-11-25 17:46:00
---------------------------------